# Example of Downloading meshes from objaverse for categories like planes or cars and generating point clouds from them

In [ ]:
import os
import signal

import open3d as o3d

# , point_cloud_generation_from_mesh
from process_msh_ply import ply2projections

import numpy as np
from tqdm import tqdm

import pandas as pd

## Filter data based on other annotations like names 

Let's load the different names annotated in the dataset and write them to a file so we can look at it:

In [ ]:
import objaverse
import multiprocessing
processes = multiprocessing.cpu_count()

In [ ]:
uids = objaverse.load_uids()
annotations = objaverse.load_annotations()
annotations[uids[0]]

In [ ]:
lvis_annotations = objaverse.load_lvis_annotations()
# get all annotation names from the dictionary
annotation_names = list(lvis_annotations.keys())

# save all annotation names to a csv file
import csv
with open("annotation_names.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(annotation_names)
    

We can select uids based on the names:

In [ ]:
# get all uids for airplane, jet_plane, fighter_jet
airplane_uids = lvis_annotations["airplane"] + lvis_annotations["jet_plane"] + lvis_annotations["fighter_jet"]
print("Number of airplane:", len(airplane_uids))

# get all uids for bicycle
bicycle_uids = lvis_annotations["bicycle"]
print("Number of bicycle:", len(bicycle_uids))

In [ ]:
# load david_holiday_cars.csv and get all the uids
df = pd.read_csv("david_holiday_cars.csv")
car_uids = df["uid"].tolist()
print("Number of cars:", len(car_uids))

example: Let's load the shapes with plane, bicycle or car associated with them.

> Note: You don't neccessarily need the annotations but you could use them to filter the data a little further.

In [ ]:
# get all car annotations
filtered_annotations = objaverse.load_annotations(uids=bicycle_uids)

# filter out the car annotations that have a face count smaller than 100 and save the new uid list
filtered_annotations = {uid: annotation for uid, annotation in filtered_annotations.items(
) if annotation["faceCount"] >= 100}
filtered_uids = list(filtered_annotations.keys())
len(filtered_annotations)

Downloading objects:

In [ ]:
# download the plane objects
downloaded_objects = objaverse.load_objects(
    uids=filtered_uids, download_processes=processes)

In [ ]:
# gives a dictionary of objects with uid as key and the path to the object as value
print(len(downloaded_objects))
downloaded_objects

In [ ]:
# save plane_objects to a csv file using pandas
df = pd.DataFrame.from_dict(downloaded_objects, orient="index")
df.to_csv("bicycle_objects.csv")

# Point cloud generation

In [ ]:
def point_cloud_generation_from_mesh(mesh_path, pc_full_path=None):
    # Load the mesh from a .glb file
    mesh = o3d.io.read_triangle_mesh(mesh_path)

    # Sample points on the mesh surface
    point_cloud = mesh.sample_points_poisson_disk(number_of_points=1000)

    if pc_full_path is not None:
        o3d.io.write_point_cloud(pc_full_path, point_cloud)

    return point_cloud
# def point_cloud_generation_from_mesh(mesh_path, pc_full_path=None, timeout=30):
#     def sample_points(mesh, pc_full_path=None):
#         # Sample points on the mesh surface
#         point_cloud = mesh.sample_points_poisson_disk(number_of_points=1000)

#         # Save the point cloud if a path is provided
#         if pc_full_path is not None:
#             o3d.io.write_point_cloud(pc_full_path, point_cloud)

#         return point_cloud
        
#     mesh = o3d.io.read_triangle_mesh(mesh_path)
#     # mesh = check_mesh_watertight_and_components(mesh)
#     if mesh is None:
#         return None

#     # Use a thread pool to manage the timeout
#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         future = executor.submit(sample_points, mesh, pc_full_path)
#         try:
#             # Wait for the result with a timeout
#             point_cloud = future.result(timeout=timeout)
#         except concurrent.futures.TimeoutError:
#             print(f"Timeout reached for {mesh_path}. Returning None.")
#             return None
#     print(f"Point cloud generated for {mesh_path}")
#     return point_cloud

In [ ]:
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException("Function call timed out")


def run_with_timeout(func, args=(), kwargs={}, timeout=10):
    # Set the signal handler for SIGALRM
    signal.signal(signal.SIGALRM, timeout_handler)
    # Set the alarm for the specified timeout
    signal.alarm(timeout)
    try:
        # Call the function with the provided arguments
        result = func(*args, **kwargs)
    except TimeoutException as e:
        # Handle the timeout exception
        print(e)
        result = None
    finally:
        # Disable the alarm
        signal.alarm(0)
    return result

def generate_point_clouds_from_class(class_name, loaded_objects):
    base_dir = class_name + "_point_clouds"
    if not os.path.exists(base_dir):
        os.makedirs(base_dir)

    # go through all plane_objects and generate point clouds
    skipped = 0
    for uid, mesh_path in tqdm(loaded_objects.items()):
        pc_full_path = os.path.join(base_dir, f"{uid}.ply")
        if os.path.exists(pc_full_path):
            continue
        
        # returned_pc = point_cloud_generation_from_mesh(mesh_path, pc_full_path)
        returned_pc = run_with_timeout(point_cloud_generation_from_mesh, args=(mesh_path, pc_full_path), timeout=30)

        if returned_pc is None:
            skipped += 1

    print(f"Skipped {skipped} objects.")

In [ ]:
class_name = "bicycle"  # "plane", "bicycle", "car"
loaded_objects = pd.read_csv(
        class_name + "_objects.csv", index_col=0).to_dict()["0"]
print(f"Generating point clouds for {len(loaded_objects)} {class_name} objects.")
generate_point_clouds_from_class(class_name, loaded_objects)

In [ ]:
class_name = "bicycle"  # "plane", "bicycle", "car"
generate_point_clouds_from_class(class_name)

In [ ]:
class_name = "car"  # "plane", "bicycle", "car"
generate_point_clouds_from_class(class_name)

## Visualization

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
base_dir = class_name + "_point_clouds"
# get all the point cloud files in the folder
pc_files = [f for f in os.listdir(base_dir) if f.endswith(".ply")]

for pc_file in pc_files:
    full_path = os.path.join(base_dir, pc_file)
    imgs = ply2projections(full_path)

    fig, axs = plt.subplots(1, 4, figsize=(15, 5))
    for i, img in enumerate(imgs):
        axs[i].imshow(img)
        axs[i].axis('off')
    plt.show()

## some further tests

In [ ]:
# Test for single mesh to point cloud
plane_uids = list(loaded_objects.keys())
selected_element = loaded_objects[plane_uids[5]]

pc_from_obj = point_cloud_generation_from_mesh(selected_element, pc_full_path="./tmp/pc_from_obj.ply")
if pc_from_obj is None:
    print("Point cloud is None.")
else:
    full_path = "./tmp/pc_from_obj.ply"
    imgs = ply2projections(full_path)

    fig, axs = plt.subplots(1, 4, figsize=(15, 5))
    for i, img in enumerate(imgs):
        axs[i].imshow(img)
        axs[i].axis('off')
    plt.show()

In [ ]:
# Test for single mesh to point cloud
plane_uids = list(loaded_objects.keys())
selected_element = loaded_objects[plane_uids[3]]

pc_from_obj = point_cloud_generation_from_mesh(
    selected_element, pc_full_path="./tmp/pc_from_obj.ply")

full_path = "./tmp/pc_from_obj.ply"
imgs = ply2projections(full_path)

fig, axs = plt.subplots(1, 4, figsize=(15, 5))
for i, img in enumerate(imgs):
    axs[i].imshow(img)
    axs[i].axis('off')
plt.show()